In [1]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K

class ModelCNN:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# first CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(32, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# second CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(512))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))

		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

In [2]:
# USAGE
# python fashion_mnist.py

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.python.keras import utils
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from imutils import build_montages
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import cv2

# initialize the number of epochs to train for, base learning rate,
# and batch size
NUM_EPOCHS = 20
INIT_LR = 1e-2
BS = 32

# initialize tensorflow session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())

# grab the Fashion MNIST dataset (if this is your first time running
# this the dataset will be automatically downloaded)
print("[INFO] loading Fashion MNIST...")
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()

# if we are using "channels first" ordering, then reshape the design
# matrix such that the matrix is:
# 	num_samples x depth x rows x columns
if K.image_data_format() == "channels_first":
	trainX = trainX.reshape((trainX.shape[0], 1, 28, 28))
	testX = testX.reshape((testX.shape[0], 1, 28, 28))
 
# otherwise, we are using "channels last" ordering, so the design
# matrix shape should be: num_samples x rows x columns x depth
else:
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))
 
# scale data to the range of [0, 1]
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

# one-hot encode the training and testing labels
trainY = utils.to_categorical(trainY, 10)
testY = utils.to_categorical(testY, 10)

# initialize the label names
labelNames = ["top", "trouser", "pullover", "dress", "coat",
	"sandal", "shirt", "sneaker", "bag", "ankle boot"]

# initialize the optimizer and model
print("[INFO] compiling model...")
opt = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / NUM_EPOCHS)
model = ModelCNN.build(width=28, height=28, depth=1, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the network
print("[INFO] training model...")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
        
H = model.fit(trainX, trainY,
	validation_data=(testX, testY),
	batch_size=BS, epochs=NUM_EPOCHS)

[INFO] loading Fashion MNIST...
[INFO] compiling model...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[INFO] training model...
Train on 60000 samples, validate on 10000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
60000/60000 [==============================] - 22s 361us/sample - loss: 0.5207 - acc: 0.8250 - val_loss: 0.3050 - val_acc: 0.8893
Epoch 2/20
60000/60000 [==============================] - 21s 347us/sample - loss: 0.3350 - acc: 0.8800 - val_loss: 0.2720 - val_acc: 0.9010
Epoch 3/20
60000/60000 [==============================] - 20s 341us/sample - loss: 0.2919 - acc: 0.8954 - val_loss: 0.2556 - val_acc: 0.9086
Epoch 4/20
60000/60000 [==============================] - 20s 339us/sample - loss: 0.2653 - acc: 0.9035 - val_loss: 0.2522 - val_acc: 0.9104
Epoch 5/20
60000/60000 [==============================] - 20s 339us/

In [3]:
# from IPython.display import clear_output, Image, display, HTML

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):

    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph
    
# def strip_consts(graph_def, max_const_size=32):
#     """Strip large constant values from graph_def."""
#     strip_def = tf.GraphDef()
#     for n0 in graph_def.node:
#         n = strip_def.node.add() 
#         n.MergeFrom(n0)
#         if n.op == 'Const':
#             tensor = n.attr['value'].tensor
#             size = len(tensor.tensor_content)
#             if size > max_const_size:
#                 tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
#     return strip_def
    
# def show_graph(graph_def, max_const_size=32):
#     """Visualize TensorFlow graph."""
#     if hasattr(graph_def, 'as_graph_def'):
#         graph_def = graph_def.as_graph_def()
#     strip_def = strip_consts(graph_def, max_const_size=max_const_size)
#     code = """
#         <script>
#           function load() {{
#             document.getElementById("{id}").pbtxt = {data};
#           }}
#         </script>
#         <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
#         <div style="height:600px">
#           <tf-graph-basic id="{id}"></tf-graph-basic>
#         </div>
#     """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
#     iframe = """
#         <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
#     """.format(code.replace('"', '&quot;'))
#     display(HTML(iframe))

import tfcoreml
import tensorflow as tf

tf.keras.backend.set_learning_phase(0) 
#Save Model to H5 File
# model.save('cnn.h5')
tf.keras.models.save_model(model, 'cnn.h5')

#Load Model from H5 File
model = tf.keras.models.load_model('cnn.h5')
global graph
graph = tf.get_default_graph()

# sess = tf.Session()
# op = sess.graph.get_operations()
# [print(m.values()) for m in op][1]

#Creating Tensorflow Pb File from Keras Model
frozen_graph = freeze_session(K.get_session(), output_names=[out.op.name for out in model.outputs])
# show_graph(frozen_graph)
tf.train.write_graph(frozen_graph, '/tmp', 'model.pb', as_text=False)

# Convert to CoreML
tfcoreml.convert(tf_model_path = '/tmp/model.pb',
                     mlmodel_path = 'cnn.mlmodel',
                     output_feature_names = ['activation_5_1/Softmax:0'])

# make predictions on the test set
# tf.keras.backend.clear_session()
with graph.as_default():
    preds = model.predict(testX)

# show a nicely formatted classification report
print("[INFO] evaluating network...")
print(classification_report(testY.argmax(axis=1), preds.argmax(axis=1),
	target_names=labelNames))

# plot the training loss and accuracy
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 116 variables.
INFO:tensorflow:Converted 116 variables to const ops.

Loading the TF graph...
Graph Loaded.
Done.
Now finding ops in the TF graph that can be dropped for inference
Now starting translation to CoreML graph.
Automatic shape interpretation succeeded for input blob conv2d_input_1:0
1/192: Analysing op name: training_2/SGD/Variable_21 ( type:  Const )
2/192: Analysing op name: training_2/SGD/Variable_20 ( type:  Const )
3/192: Analysing op name: training_2/SGD/Variable_19 ( type:  Const )
4/192: Analysing op name: training_2/SGD/Variable_18 ( type:  Const )
5/192: Analysing op name: training_2/SGD/Variable_17 ( type:  Const )
6/192: Analysing op name: training_2/SGD/Variable_16 ( type:  Const )
7/192: Analysing op name: training_2/SGD/Variable_15 ( type:  Const )
8/192: Analysing op name: training

127/192: Analysing op name: dense_2/BiasAdd ( type:  BiasAdd )
128/192: Analysing op name: activation_4_1/Relu ( type:  Relu )
129/192: Analysing op name: batch_normalization_v1_4_1/batchnorm/mul_1 ( type:  Mul )
130/192: Analysing op name: batch_normalization_v1_4_1/batchnorm/add_1 ( type:  Add )
131/192: Analysing op name: dropout_2_1/Identity ( type:  Identity )
132/192: Analysing op name: dense_1_1/MatMul ( type:  MatMul )
133/192: Analysing op name: dense_1_1/BiasAdd ( type:  BiasAdd )
134/192: Analysing op name: activation_5_1/Softmax ( type:  Softmax )
135/192: Analysing op name: training/SGD/Variable_21 ( type:  Const )
Translation to CoreML spec completed. Now compiling and saving the CoreML model.

 Core ML model generated. Saved at location: cnn.mlmodel 

Core ML input(s): 
 [name: "conv2d_input_1__0"
type {
  multiArrayType {
    shape: 1
    shape: 28
    shape: 28
    dataType: DOUBLE
  }
}
]
Core ML output(s): 
 [name: "activation_5_1__Softmax__0"
type {
  multiArrayType